In [1]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from PIL import Image
from IPython.display import Image, display
from sklearn.datasets import load_iris

In [2]:
def x_round(x, numpixels = 1):
    # round value to nearest pixel (one pixel is 1/50 A)
    return round(x*50/numpixels)/(50/numpixels)

In [3]:
directory = '/mnt_home/zoek/code/APF-BL-DAP/Zoe/6sResults/'
spectra_file = directory + 's_detectionWLs.csv'
residuals_file = directory + 'r_detectionWLs.csv'

spectra_results = pd.read_csv(spectra_file)
residuals_results = pd.read_csv(residuals_file)

persistent_spectra = pd.read_csv(directory + 's_persistent_events.csv')
persistent_residuals = pd.read_csv(directory + 'r_persistent_events.csv')

In [4]:
# repeated wavelengths using the 3 AMD run - in the rest frame and observed frame

rep_observed = pd.read_csv('/mnt_home/zoek/code/APF-BL-DAP/Zoe/3sResults/rep_observed_wls.csv')
rep_rest = pd.read_csv('/mnt_home/zoek/code/APF-BL-DAP/Zoe/3sResults/rep_rest_wls.csv')

rep_observed_wls_3s = rep_observed['observed wl']
rep_rest_wls_3s = rep_rest['rest wl']

rep_observed_wls_3s = x_round(rep_observed_wls_3s)
rep_rest_wls_3s = x_round(rep_rest_wls_3s)

In [5]:
# These are the 55 events that passed all the tests.
persistent_residuals

,Unnamed: 0,star,identifier,r shifted wl,r unshifted wl,r heights,r AMD heights,rounded r shifted wl,observation time,rounded r unshifted wl
0,135,HIP84862,HIP84862_6,5001.940428,5001.940428,0.166277,6.047927,5001.94,2019-07-10T05:01:52.82,5001.94
1,133,HIP84862,HIP84862_5,5001.940428,5003.346098,0.189115,6.386022,5001.94,2019-07-10T04:59:03.60,5003.34
2,132,HIP84862,HIP84862_3,5001.940428,5001.788053,0.192912,6.297325,5001.94,2019-07-10T04:53:32.98,5001.78
3,138,HIP84862,HIP84862_7,5001.940428,5001.940428,0.198079,6.527165,5001.94,2019-07-10T05:03:43.25,5001.94
4,15,HIP14286,HIP14286_3,5004.504441,5004.504441,0.132577,6.454437,5004.50,2018-11-05T08:02:23.48,5004.50
...,...,...,...,...,...,...,...,...,...,...
168,11,HIP117463,HIP117463_8,5895.866519,5895.866519,0.427691,8.259848,5895.86,2019-11-06T08:50:05.64,5895.86
169,10,HIP117463,HIP117463_10,5895.866519,5895.866519,0.345801,6.869269,5895.86,2019-11-06T09:01:34.06,5895.86
170,9,HIP114430,HIP114430_12,5896.526877,5896.526877,0.245833,12.548018,5896.52,2018-04-02T12:32:30.48,5896.52
171,8,HIP114430,HIP114430_1,5896.526877,5897.365278,0.151014,7.317310,5896.52,2018-04-02T12:48:01.11,5897.36


## Repeated observed wavelengths - sky emission lines, deblazing artifacts

In [6]:
# Remove all candidates with observed wavelengths that fall into rep_observed_wls_3s
persistent_residuals_rem_obs = persistent_residuals[~persistent_residuals['rounded r unshifted wl'].isin(rep_observed_wls_3s)]
persistent_residuals_rem_obs.reset_index(inplace=True)

In [7]:
# These are the 22 TOTAL observed wavelengths that are removed
removed_obs_wls = persistent_residuals[persistent_residuals['rounded r unshifted wl'].isin(rep_observed_wls_3s)]
removed_obs_wls = removed_obs_wls['rounded r unshifted wl'].unique()
print(len(removed_obs_wls))
removed_obs_wls

22


array([5001.94, 5012.  , 5014.16, 5017.54, 5018.38, 5049.88, 5049.92,
       5083.26, 5107.36, 5166.22, 5168.82, 5168.86, 5191.54, 5226.46,
       5226.76, 5227.08, 5371.6 , 5405.66, 5405.72, 5853.76, 5853.78,
       5895.76])

In [20]:
# These are the 13 wavelengths that are removed that are not from deblazing artifacts 
print(len(removed_obs_wls[removed_obs_wls > 5110]))
np.sort(removed_obs_wls[removed_obs_wls > 5110])

13


array([5166.22, 5168.82, 5168.86, 5191.54, 5226.46, 5226.76, 5227.08,
       5371.6 , 5405.66, 5405.72, 5853.76, 5853.78, 5895.76])

In [9]:
# drop all candidates that have less than 3 observations in a night
less_than_3 = persistent_residuals_rem_obs.groupby(['rounded r shifted wl', 'star']).count().reset_index()
less_than_3 = less_than_3[less_than_3['identifier'] < 3]
less_than_3.reset_index(inplace=True)
less_than_3 = less_than_3.loc[:, ['rounded r shifted wl', 'star']]
for index, row in less_than_3.iterrows():
    wl = row['rounded r shifted wl']
    star = row['star']
    sub_table = persistent_residuals_rem_obs[persistent_residuals_rem_obs['star'] == star]
    sub_table = sub_table[sub_table['rounded r shifted wl'] == wl]
    indicies_to_remove = list(sub_table['index'])
    persistent_residuals_rem_obs = persistent_residuals_rem_obs[~persistent_residuals_rem_obs['index'].isin(indicies_to_remove)]

In [10]:
# These are all the events after removing the observed wavelengths that show up repeatedly across the 3s results.
# 28 unique events
persistent_residuals_rem_obs

,index,Unnamed: 0,star,identifier,r shifted wl,r unshifted wl,r heights,r AMD heights,rounded r shifted wl,observation time,rounded r unshifted wl
2,4,15,HIP14286,HIP14286_3,5004.504441,5004.504441,0.132577,6.454437,5004.50,2018-11-05T08:02:23.48,5004.50
3,5,14,HIP14286,HIP14286_2,5004.504441,5004.385866,0.133850,6.492382,5004.50,2018-11-05T08:08:28.93,5004.38
4,6,13,HIP14286,HIP14286_1,5004.504441,5004.487502,0.129181,6.790105,5004.50,2018-11-05T07:56:10.17,5004.48
9,17,63,HIP68030,HIP68030_2,5014.195018,5013.609535,0.108921,6.402652,5014.20,2017-05-14T06:36:48.60,5013.60
10,18,50,HIP68030,HIP68030_0,5014.195018,5014.195018,0.110437,6.167016,5014.20,2017-05-14T06:29:09.62,5014.20
...,...,...,...,...,...,...,...,...,...,...,...
104,168,11,HIP117463,HIP117463_8,5895.866519,5895.866519,0.427691,8.259848,5895.86,2019-11-06T08:50:05.64,5895.86
105,169,10,HIP117463,HIP117463_10,5895.866519,5895.866519,0.345801,6.869269,5895.86,2019-11-06T09:01:34.06,5895.86
106,170,9,HIP114430,HIP114430_12,5896.526877,5896.526877,0.245833,12.548018,5896.52,2018-04-02T12:32:30.48,5896.52
107,171,8,HIP114430,HIP114430_1,5896.526877,5897.365278,0.151014,7.317310,5896.52,2018-04-02T12:48:01.11,5897.36


In [11]:
print('# final events')
len(persistent_residuals_rem_obs.groupby(['star', 'rounded r shifted wl']).first().reset_index())

# final events


28

In [12]:
print('# stars')
len(persistent_residuals_rem_obs['star'].unique())

# stars


20

In [13]:
stars = list(persistent_residuals_rem_obs['star'].unique())

In [14]:
star = 'HIP68030'

In [15]:
persistent_residuals_rem_obs[persistent_residuals_rem_obs['star'] == star]

,index,Unnamed: 0,star,identifier,r shifted wl,r unshifted wl,r heights,r AMD heights,rounded r shifted wl,observation time,rounded r unshifted wl
9,17,63,HIP68030,HIP68030_2,5014.195018,5013.609535,0.108921,6.402652,5014.20,2017-05-14T06:36:48.60,5013.60
10,18,50,HIP68030,HIP68030_0,5014.195018,5014.195018,0.110437,6.167016,5014.20,2017-05-14T06:29:09.62,5014.20
11,19,54,HIP68030,HIP68030_1,5014.195018,5013.906502,0.107112,6.721232,5014.20,2017-05-14T06:21:30.41,5013.90
19,38,52,HIP68030,HIP68030_0,5110.334789,5110.334789,0.122762,6.454940,5110.34,2017-05-14T06:29:09.62,5110.34
20,39,61,HIP68030,HIP68030_2,5110.334789,5109.738080,0.116781,6.304794,5110.34,2017-05-14T06:36:48.60,5109.74
21,40,56,HIP68030,HIP68030_1,5110.334789,5110.040741,0.121442,7.228144,5110.34,2017-05-14T06:21:30.41,5110.04
54,104,87,HIP68030,HIP68030_7,5405.679846,5405.679846,0.105322,8.323868,5405.68,2020-06-05T05:51:38.20,5405.68
55,105,80,HIP68030,HIP68030_6,5405.679846,5405.679846,0.148745,7.980924,5405.68,2020-06-05T05:38:57.75,5405.68
56,106,95,HIP68030,HIP68030_8,5405.679846,5405.679846,0.150350,7.983830,5405.68,2020-06-05T05:45:14.14,5405.68
66,118,67,HIP68030,HIP68030_3,5476.812997,5475.348854,0.120124,6.631612,5476.82,2020-05-29T05:26:57.22,5475.34
